# Initialisation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('seaborn')

import ta
import plotly
import plotly.express as px
import plotly.graph_objects as go
import cufflinks as cf

from datetime import datetime
from arch import arch_model

In [2]:
from scipy.stats import linregress

In [3]:
# Load the raw data
df_l = pd.read_csv('./FX Data/GBPUSD.csv', names=['date','open','high','low','close'])

In [4]:
df = df_l[0:500]

# Charts

In [6]:
#fig_cdls = go.Figure(data=[go.Candlestick(x=df['Date'][0:100],open=df['Open'],high=df['High'],low=df['Low'],close=df['Close'])])
#fig_cdls.show()

In [ ]:
#fig = px.line(df[0:500],x='Date',y='Close')
#fig.show()

In [ ]:
# ind_EMA = ta.trend.EMAIndicator(close=df['Close'],n=10,fillna=False)

# df['EMA'] = ind_EMA.ema_indicator()

In [ ]:
# fig_ind = go.Figure()

# fig_ind.update_layout(xaxis_range=[0,50],
#                       yaxis_range=(1.5,1.7),
#                  title_text = "Close and EMA")

# fig_ind.add_trace(go.Scatter(
#                     x=df['Date'],
#                     y=df['Close'],
#                     name='Close',
#                     line_color='dimgray',
#                     opacity=0.4))

# fig_ind.add_trace(go.Scatter(
#                     x=df['Date'],
#                     y=df['EMA'],
#                     name='EMA',
#                     line_color='deepskyblue',
#                     opacity=0.4))

# fig_ind.show()

# Adding features
## Technical Analysis features

In [7]:
df['wclose'] = (df['close']*2+df['high']+df['low'])/4

In [8]:
for i in range(1,len(df)):
    if df.at[i,'high'] > df.at[i-1,'high']:
        df.at[i,'delta'] = 1
    else:
        df.at[i,'delta'] = 0

In [37]:
def add_SO(i):
    ind_SO = ta.momentum.StochasticOscillator(high=df['high'],low=df['low'],close=df['close'],n=i)
    name = 'so_%s' %(i)
    df[name] = ind_SO.stoch()
    return

for i in [3,4,5,8,9,10]:
    add_SO(i)

In [41]:
def add_WR(i):
    ind_WR = ta.momentum.WilliamsRIndicator(high=df['high'],low=df['low'],close=df['close'],lbp=i)
    name = 'wr_%s' %(i)
    df[name] = ind_WR.wr()
    return

for i in [6,7,8,9,10]:
    add_WR(i)

In [47]:
def add_ROC(i):
    ind_ROC = ta.momentum.ROCIndicator(close=df['close'],n=i)
    name = 'roc_%s' %(i)
    df[name] = ind_ROC.roc()
    return

for i in [12,13,14,15]:
    add_ROC(i)

In [ ]:
def add_WCP(i):
    ind_WCP = ta.momentum.EMAIndicator(close=df['wclose'],n=i)
    name = 'wcp_%s' %(i)
    df[name] = ind_WCP.ema_indicator()
    return

add_WCP(15)

In [52]:
def add_MACD(i,j):
    ind_MACD = ta.trend.MACD(close=df['close'],n_fast=i,n_slow=j)
    name ='macd_%s_%s' %(i,j)
    df[name] = ind_MACD.macd()
    return

add_MACD(15,30)  

In [61]:
def add_CCI(i):
    ind_CCI = ta.trend.cci(high=df['high'],low=df['low'],close=df['close'],n=i)
    name = 'cci_%s' %(i)
    df[name] = ind_CCI
    return

add_CCI(15)

## Signal Processing features

In [115]:
for i in range(1,len(df)):
    df.at[i,'hi_avg_2'] =  (df.at[i-1,'high'] + df.at[i,'high'])/2
    df.at[i,'lo_avg_2'] =  (df.at[i-1,'low'] + df.at[i,'low'])/2
    df.at[i,'hilo_avg_2'] = (df.at[i,'hi_avg_2'] + df.at[i,'lo_avg_2'])/2
    df.at[i,'hilo_avg'] = (df.at[i,'high'] + df.at[i,'low'])/2

In [148]:
def add_slope(i):
    for j in range(i-1,len(df)):
        a=df['high'][j-(i-1):j+1]
        b=[]
        for k in range(i):
            b.append(k)
        name = 'slope_%s' %(i)
        df.at[j,name] = linregress(a,b).slope
    
for i in [3,4,5,10,20,30]:
    add_slope(i)

In [9]:
df.head()

,date,open,high,low,close,wclose,delta
0,3/1/2000,1.6147,1.6396,1.6145,1.6358,1.631425,NaN
1,4/1/2000,1.6358,1.6413,1.6316,1.6373,1.636875,1.0
2,5/1/2000,1.6373,1.6451,1.6352,1.6419,1.641025,1.0
3,6/1/2000,1.6419,1.6509,1.6412,1.6477,1.646875,1.0
4,7/1/2000,1.6477,1.6498,1.6361,1.6388,1.640875,0.0


# Classifiers
## RBF Classifier